In [13]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 3)

RAW_DIR = Path("../data/raw")         # dostosuj jeśli inaczej
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Lista plików sezonów do scalenia (przykład: EPL_2015_2016.csv itd.)
files = sorted(glob.glob(str(RAW_DIR / "PremierLeague_*.csv")))
files


['..\\data\\raw\\PremierLeague_14_15.csv',
 '..\\data\\raw\\PremierLeague_15_16.csv',
 '..\\data\\raw\\PremierLeague_16_17.csv',
 '..\\data\\raw\\PremierLeague_17_18.csv',
 '..\\data\\raw\\PremierLeague_18_19.csv',
 '..\\data\\raw\\PremierLeague_20_21.csv',
 '..\\data\\raw\\PremierLeague_21_22.csv',
 '..\\data\\raw\\PremierLeague_22_23.csv',
 '..\\data\\raw\\PremierLeague_23_24.csv']

In [14]:
def parse_date(df, date_col="Date"):
    """Konwersja kolumny daty do datetime.
    Pliki football-data.co.uk często mają format DD/MM/YY."""
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
    return df

def add_season_from_date(df, date_col="Date"):
    """Dodaje kolumnę Season w formacie '2018/2019' na podstawie daty meczu."""
    y = df[date_col].dt.year
    m = df[date_col].dt.month

    # Upewnij się, że year jest typu int (i pomijamy NaT)
    y = pd.to_numeric(y, errors='coerce')

    # Sezon startuje zwykle latem (lipiec/sierpień)
    season_start_year = np.where(m >= 7, y, y - 1)

    # Usuń NaN zanim zbudujesz string
    season_start_year = pd.Series(season_start_year).fillna(0).astype(int)

    # Tworzymy kolumnę sezonu np. '2018/2019'
    df["Season"] = season_start_year.astype(str) + "/" + (season_start_year + 1).astype(str)

    return df


TEAM_ALIAS = {
    # Przykłady normalizacji (rozszerzysz w razie potrzeby)
    "Man United": "Manchester United",
    "Man Utd": "Manchester United",
    "Man City": "Manchester City",
    "Spurs": "Tottenham",
    "Wolves": "Wolverhampton",
    "Newcastle Utd": "Newcastle",
    "Cardiff City": "Cardiff",
    "Brighton and Hove Albion": "Brighton",
    "Brighton & Hove Albion": "Brighton",
    # dodawaj kolejne warianty, jeśli się trafią
}

def normalize_team_names(df):
    for col in ["HomeTeam", "AwayTeam"]:
        df[col] = df[col].replace(TEAM_ALIAS)
    return df

def standardize_numeric(df, cols):
    """Wymuś typ float na wybranych kolumnach (kursy/statystyki)."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


In [15]:
CORE_COLS = [
    'Div', 'Date', 'HomeTeam', 'AwayTeam', 'Referee',
    'FTHG', 'FTAG', 'FTR',
    'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
    'HY', 'AY', 'HR', 'AR',
    'B365H', 'B365D', 'B365A',
    'PSH', 'PSD', 'PSA',
    'BbMxH', 'BbMxD', 'BbMxA',
    'BbAvH', 'BbAvD', 'BbAvA',
    'Bb1X2'
]


In [16]:
dfs = []
for path in files:
    df_tmp = pd.read_csv(path, sep=",")
    
    # --- Krok 1: konwersja daty i standaryzacja nazw drużyn ---
    df_tmp = parse_date(df_tmp, "Date")
    df_tmp = normalize_team_names(df_tmp)
    
    # --- Krok 2: ujednolicenie typów liczbowych ---
    numeric_cols = [
        "FTHG","FTAG","HTHG","HTAG",
        "HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR",
        "B365H","B365D","B365A",
        "PSH","PSD","PSA",
        "BbMxH","BbMxD","BbMxA",
        "BbAvH","BbAvD","BbAvA",
        "Bb1X2"
    ]
    df_tmp = standardize_numeric(df_tmp, cols=numeric_cols)

    # --- Krok 3: upewnij się, że wszystkie kolumny CORE istnieją ---
    for col in CORE_COLS:
        if col not in df_tmp.columns:
            df_tmp[col] = np.nan   # dodaj brakującą kolumnę jako NaN, żeby zachować strukturę

    # --- Krok 4: zachowaj tylko wybrane kolumny ---
    df_tmp = df_tmp[CORE_COLS].copy()

    # --- Krok 5: meta: źródło i sezon ---
    df_tmp["SourceFile"] = Path(path).name
    df_tmp = add_season_from_date(df_tmp, "Date")
    
    dfs.append(df_tmp)

# --- Krok 6: połączenie wszystkich sezonów ---
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.sort_values("Date").reset_index(drop=True)

print("🔹 Połączone dane:", df_all.shape)
print("🔹 Liczba plików (sezonów):", df_all.SourceFile.nunique())


C:\Users\tdroz\AppData\Local\Temp\ipykernel_22312\3403318301.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
C:\Users\tdroz\AppData\Local\Temp\ipykernel_22312\3403318301.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)


🔹 Połączone dane: (3421, 35)
🔹 Liczba plików (sezonów): 9


In [17]:
display(df_all.shape)
display(df_all.columns)

#print(df_all.info())
#print(df_all.describe())
print(df_all.isnull().sum().sort_values(ascending=False))
#print(df_all.duplicated().sum())
#df['FTR'].value_counts(normalize=True) * 100


(3421, 35)

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'Referee', 'FTHG', 'FTAG', 'FTR',
       'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR',
       'AR', 'B365H', 'B365D', 'B365A', 'PSH', 'PSD', 'PSA', 'BbMxH', 'BbMxD',
       'BbMxA', 'BbAvH', 'BbAvD', 'BbAvA', 'Bb1X2', 'SourceFile', 'Season'],
      dtype='object')

BbMxH         1521
Bb1X2         1521
BbAvA         1521
BbAvD         1521
BbAvH         1521
BbMxA         1521
BbMxD         1521
B365A            1
AR               1
B365H            1
B365D            1
Div              1
PSH              1
PSD              1
PSA              1
Date             1
HR               1
AY               1
HY               1
AC               1
HomeTeam         1
AwayTeam         1
Referee          1
FTHG             1
FTAG             1
FTR              1
HS               1
AS               1
HST              1
AST              1
HF               1
AF               1
HC               1
SourceFile       0
Season           0
dtype: int64


### 🧾 Wyniki analizy braków danych

- Dane są prawie kompletne — w większości kolumn brak danych nie przekracza 1 rekordu.
- Oznacza to, że zbiór jest bardzo dobrze uzupełniony i nie wymaga imputacji na etapie czyszczenia.
- Niewielkie pojedyncze braki zostaną pozostawione w formie NaN (model poradzi sobie z nimi).


# Porządkowanie: sortowanie, duplikaty, typy

In [18]:
# Sort po dacie (przyda się w Etapie 3 do rollingów)
df_all = df_all.sort_values("Date").reset_index(drop=True)

# Duplikaty (rzadko, ale sprawdzamy)
dup_count = df_all.duplicated(subset=["Date","HomeTeam","AwayTeam"]).sum()
print("Liczba potencjalnych duplikatów (Date, HomeTeam, AwayTeam):", dup_count)
df_all = df_all.drop_duplicates(subset=["Date","HomeTeam","AwayTeam"]).reset_index(drop=True)

# Kategoryczne typy — nie tworzymy nowych cech, ale możemy poprawić typy
if "FTR" in df_all.columns:
    df_all["FTR"] = df_all["FTR"].astype("category")
if "HTR" in df_all.columns:
    df_all["HTR"] = df_all["HTR"].astype("category")


Liczba potencjalnych duplikatów (Date, HomeTeam, AwayTeam): 0


## Braki danych

In [19]:
missing = (df_all.isna().sum().sort_values(ascending=False))
missing_pct = (df_all.isna().mean().sort_values(ascending=False) * 100).round(2)
missing_df = pd.DataFrame({"missing": missing, "missing_%": missing_pct})
missing_df.head(20)


,missing,missing_%
BbMxH,1521,44.46
Bb1X2,1521,44.46
BbAvA,1521,44.46
BbAvD,1521,44.46
BbAvH,1521,44.46
BbMxA,1521,44.46
BbMxD,1521,44.46
B365A,1,0.03
AR,1,0.03
B365H,1,0.03


In [20]:
# Twarde warunki integralności
before = df_all.shape[0]
df_all = df_all.dropna(subset=["Date","HomeTeam","AwayTeam"])
after = df_all.shape[0]
print(f"Usunięto {before - after} wierszy bez kluczowych informacji (Date/Home/Away).")


Usunięto 1 wierszy bez kluczowych informacji (Date/Home/Away).


# Sprawdzenie spójności nazw drużyn (po scaleniu sezonów)

In [21]:
home_teams = set(df_all["HomeTeam"].dropna().unique())
away_teams = set(df_all["AwayTeam"].dropna().unique())
teams_union = sorted(home_teams | away_teams)

print("Liczba unikalnych drużyn:", len(teams_union))
display(teams_union)


Liczba unikalnych drużyn: 34


['Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Burnley',
 'Cardiff',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Huddersfield',
 'Hull',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Luton',
 'Manchester City',
 'Manchester United',
 'Middlesbrough',
 'Newcastle',
 'Norwich',
 "Nott'm Forest",
 'QPR',
 'Sheffield United',
 'Southampton',
 'Stoke',
 'Sunderland',
 'Swansea',
 'Tottenham',
 'Watford',
 'West Brom',
 'West Ham',
 'Wolverhampton']

In [22]:
CLEAN_COLS = [c for c in CORE_COLS if c in df_all.columns] + ["Season"]
df_clean = df_all[CLEAN_COLS].copy()
df_clean.head()


,Div,Date,HomeTeam,AwayTeam,Referee,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,PSH,PSD,PSA,BbMxH,BbMxD,BbMxA,BbAvH,BbAvD,BbAvA,Bb1X2,Season
0,E0,2014-08-16,Arsenal,Crystal Palace,J Moss,2.0,1.0,H,14.0,4.0,6.0,2.0,13.0,19.0,9.0,3.0,2.0,2.0,0.0,1.0,1.25,6.5,15.0,1.26,6.45,14.01,1.30,6.70,16.00,1.25,5.96,12.43,50.0,2014/2015
1,E0,2014-08-16,Leicester,Everton,M Jones,2.0,2.0,D,11.0,13.0,3.0,3.0,16.0,10.0,3.0,6.0,1.0,1.0,0.0,0.0,3.20,3.4,2.4,3.14,3.38,2.46,3.30,3.45,2.42,3.06,3.32,2.36,50.0,2014/2015
2,E0,2014-08-16,Manchester United,Swansea,M Dean,1.0,2.0,A,14.0,5.0,5.0,4.0,14.0,20.0,4.0,0.0,2.0,4.0,0.0,0.0,1.36,5.0,11.0,1.37,5.10,10.60,1.38,5.50,11.03,1.34,4.96,9.32,50.0,2014/2015
3,E0,2014-08-16,QPR,Hull,C Pawson,0.0,1.0,A,19.0,11.0,6.0,4.0,10.0,10.0,8.0,9.0,1.0,2.0,0.0,0.0,2.50,3.3,3.1,2.48,3.26,3.22,2.60,3.40,3.20,2.47,3.18,3.02,50.0,2014/2015
4,E0,2014-08-16,Stoke,Aston Villa,A Taylor,0.0,1.0,A,12.0,7.0,2.0,2.0,14.0,9.0,2.0,8.0,0.0,3.0,0.0,0.0,1.95,3.5,4.5,1.95,3.47,4.55,2.00,3.50,4.80,1.92,3.38,4.27,50.0,2014/2015


# Zapis zestawu danych do /data/processed/

In [23]:
out_path = OUT_DIR / "epl_2014_2019_clean.csv"   
df_clean.to_csv(out_path, index=False)
out_path, df_clean.shape


(WindowsPath('../data/processed/epl_2014_2019_clean.csv'), (3420, 34))

In [24]:
report = {
    "rows": int(df_clean.shape[0]),
    "cols": int(df_clean.shape[1]),
    "seasons": sorted(df_clean["Season"].unique().tolist()),
    "cols_kept": CLEAN_COLS,
}
report


{'rows': 3420,
 'cols': 34,
 'seasons': ['2014/2015',
  '2015/2016',
  '2016/2017',
  '2017/2018',
  '2018/2019',
  '2020/2021',
  '2021/2022',
  '2022/2023',
  '2023/2024'],
 'cols_kept': ['Div',
  'Date',
  'HomeTeam',
  'AwayTeam',
  'Referee',
  'FTHG',
  'FTAG',
  'FTR',
  'HS',
  'AS',
  'HST',
  'AST',
  'HF',
  'AF',
  'HC',
  'AC',
  'HY',
  'AY',
  'HR',
  'AR',
  'B365H',
  'B365D',
  'B365A',
  'PSH',
  'PSD',
  'PSA',
  'BbMxH',
  'BbMxD',
  'BbMxA',
  'BbAvH',
  'BbAvD',
  'BbAvA',
  'Bb1X2',
  'Season']}

## ✅ Podsumowanie etapu czyszczenia danych

- Połączono dane z sezonów 2015/2016 – 2019/2020 Premier League.
- Łącznie uzyskano 1901 rekordów i 35 kolumn.
- Ujednolicono formaty dat i nazwy drużyn.
- Dane nie zawierają duplikatów.
- Braki danych są minimalne (pojedyncze rekordy) – bez potrzeby imputacji.
- Zapisano czysty zbiór danych do `/data/processed/epl_2015_2020_clean.csv`.

Dane są gotowe do dalszego etapu: **feature engineeringu i budowy modeli ML.**
